In [13]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [14]:
r=requests.get("https://dumps.wikimedia.org/other/geoeditors/")

soup = BeautifulSoup(r.content, 'html.parser')
headers = ["wiki", "country", "activity_level", "lower_bound", "upper_bound", "month"]
tsvFile=[]
for link in soup.find_all('a'):
    if link.get('href')[0:18] == "geoeditors-monthly":
        url="https://dumps.wikimedia.org/other/geoeditors/{}".format(link.get('href'))
        urllib.request.urlretrieve(url, './dataDump/{}'.format(link.get('href')))
        df = pd.read_csv("dataDump/{}".format(link.get('href')), sep='\t', names=headers)
        df["month"]=link.get('href')[19:26]
        tsvFile.append(df)
        

In [15]:
merged_df = pd.concat(tsvFile)
merged_df['year'] = pd.to_datetime(merged_df.month, format='%Y-%m')
merged_df['year'] = merged_df['year'].apply(lambda x: x.year)

merged_df.to_csv('master.csv', index=False)

In [16]:
merged_df.query("""(year == 2018) & (country == 'India') & (wiki == 'enwiki')""").groupby(['month', 'year', 'country']).agg({'lower_bound': 'sum', 'upper_bound': 'sum'})

,,,lower_bound,upper_bound
month,year,country,,
2018-01,2018,India,4902,4920
2018-02,2018,India,4392,4410
2018-03,2018,India,5032,5050
2018-04,2018,India,4932,4950
2018-05,2018,India,4822,4840
2018-06,2018,India,4832,4850
2018-07,2018,India,4942,4960
2018-08,2018,India,5162,5180
2018-09,2018,India,4902,4920


In [17]:
agg1 = merged_df.groupby(['month', 'year', 'country', 'wiki']).agg({'lower_bound': 'sum', 'upper_bound': 'sum'})
agg2 = agg1.groupby(['year', 'country', 'wiki']).agg({'lower_bound': 'mean', 'upper_bound': 'mean'}).astype({'lower_bound': int, 'upper_bound': int})

agg2.to_csv('masterData.csv', index=True)


In [18]:
agg2

lower_bound  upper_bound
year country       wiki                                
2018 Albania       alswiki               1           10
                   bgwiki                1           10
                   bswiki                1           10
                   dewiki                1           15
                   elwiki                1           10
...                                    ...          ...
2023 unknown       zh_yuewiki            1           10
                   zhwiki               32           50
                   zuwiki                1           10
     Åland Islands enwiki                1           10
                   svwiki                1           10

[36033 rows x 2 columns]